In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [ ]:
import pandas as pd

class VaersDescrReader:
    
    def __init__(self, dataDir):
        self.dataDir = dataDir        

    def readAllVaersDescrs(self):
        return self.readVaersDescrs(["2021", "2022"])
        
    def readVaersDescrs(self, years):
        return [self.readVaersDescr(year) for year in years]

    def readVaersDescr(self, year):
        folder = self.dataDir + "/" + year + "VAERSData/"
        return {
            'VAERSDATA':
                self._read_csv(
                    folder + year + "VAERSDATA.csv",
                    # FK-TODO: use Column enum
                    ['VAERS_ID', 'DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT']),
            'VAERSVAX':
                    self._read_csv(
                        folder + year + "VAERSVAX.csv",
                        ['VAERS_ID', 'VAX_DOSE_SERIES', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT'],
                        dtype = {"VAX_DOSE_SERIES": "string"})
            }

    def _read_csv(self, file, usecols, dtype = {}):
        return pd.read_csv(
            file,
            index_col = 'VAERS_ID',
            encoding = 'latin1',
            low_memory = False,
            usecols = usecols,
            dtype = dtype)


In [ ]:
import pandas as pd

class VaersDescr2DataFrameConverter:

    @staticmethod
    def createDataFrameFromDescr(vaersDescr):
        return pd.merge(
                vaersDescr['VAERSDATA'],
                vaersDescr['VAERSVAX'],
                how = 'left',
                left_index = True,
                right_index = True,
                validate = 'one_to_many')

    @staticmethod
    def createDataFrameFromDescrs(vaersDescrs):
        dataFrames = [VaersDescr2DataFrameConverter.createDataFrameFromDescr(vaersDescr) for vaersDescr in vaersDescrs]
        return pd.concat(dataFrames)


In [ ]:
import pandas as pd

class DataFrameFilter:
    
    def __init__(self, dataFrame):
        self.dataFrame = dataFrame        

    def filterBy(self, manufacturer = None, dose = None):
        return self.dataFrame[self._isCovid19() & self._isManufacturer(manufacturer) & self._isDose(dose)]

    def filterForSevereEffects(self, dose):
        return self.filterBy(dose = dose)

    def _isCovid19(self):
        return self.dataFrame["VAX_TYPE"] == "COVID19"

    def _isManufacturer(self, manufacturer):
        return self.dataFrame["VAX_MANU"] == manufacturer if manufacturer is not None else True

    def _isDose(self, dose):
        return self.dataFrame["VAX_DOSE_SERIES"].str.contains(dose) if dose is not None else True


In [ ]:
import pandas as pd

class BatchCodeTableFactory:

    def __init__(self, dataFrame : pd.DataFrame):
        self.dataFrame = dataFrame        

    def createBatchCodeTable(self, manufacturer, dose):
        df = DataFrameFilter(self.dataFrame).filterBy(manufacturer = manufacturer, dose = dose)
        return BatchCodeTableFactory._asDataFrame(
            {
                'ADRs': BatchCodeTableFactory._getADRs(df),
                'DEATHS': BatchCodeTableFactory._getDEATHS(df),
                'DISABILITIES': BatchCodeTableFactory._getDISABILITIES(df),
                'LIFE THREATENING ILLNESSES': BatchCodeTableFactory._getLIFE_THREATENING_ILLNESSES(df)
            })

    # create table from https://www.howbadismybatch.com/combined.html
    def createSevereEffectsBatchCodeTable(self, dose):
        df = DataFrameFilter(self.dataFrame).filterForSevereEffects(dose)
        return BatchCodeTableFactory._addCompanyColumn(
            BatchCodeTableFactory._asDataFrame(
                {
                    'ADRs': BatchCodeTableFactory._getADRs(df),
                    'DEATHS': BatchCodeTableFactory._getDEATHS(df),
                    'DISABILITIES': BatchCodeTableFactory._getDISABILITIES(df),
                    'LIFE THREATENING ILLNESSES': BatchCodeTableFactory._getLIFE_THREATENING_ILLNESSES(df),
                    'HOSPITALISATIONS': BatchCodeTableFactory._getHOSPITALISATIONS(df),
                    'EMERGENCY ROOM OR DOCTOR VISITS': BatchCodeTableFactory._getER_VISITs(df)
                }),
            BatchCodeTableFactory._createCompanyByBatchCodeTable(df))

    @staticmethod
    def _getADRs(df):
        return df['VAX_LOT'].value_counts()

    @staticmethod
    def _getDEATHS(df):
        return BatchCodeTableFactory._countValues(df, 'DIED')

    @staticmethod
    def _getDISABILITIES(df):
        return BatchCodeTableFactory._countValues(df, 'DISABLE')

    @staticmethod
    def _getLIFE_THREATENING_ILLNESSES(df):
        return BatchCodeTableFactory._countValues(df, 'L_THREAT')

    @staticmethod
    def _getHOSPITALISATIONS(df):
        return BatchCodeTableFactory._countValues(df, 'HOSPITAL')

    @staticmethod
    def _getER_VISITs(df):
        return BatchCodeTableFactory._countValues(df, 'ER_VISIT')

    @staticmethod
    def _countValues(df, column):
        return df[df[column] == 'Y']['VAX_LOT'].value_counts()

    @staticmethod
    def _asDataFrame(dict):
        dataFrame = pd.concat(dict, axis = 'columns')
        dataFrame.index.name = 'VAX_LOT'
        return dataFrame.replace(to_replace = np.nan, value = 0)

    @staticmethod
    def _addCompanyColumn(batchCodeTable, companyByBatchCodeTable):
        return pd.merge(
            batchCodeTable,
            companyByBatchCodeTable,
            how = 'left',
            left_index = True,
            right_index = True,
            validate = 'one_to_one')

    @staticmethod
    def _createManufacturerByBatchCodeTable(df):
        manufacturerByBatchCodeTable = df[['VAX_LOT', 'VAX_MANU']]
        manufacturerByBatchCodeTable = manufacturerByBatchCodeTable.drop_duplicates(subset = ['VAX_LOT'])
        return manufacturerByBatchCodeTable.set_index('VAX_LOT')

    @staticmethod
    def _createCompanyByBatchCodeTable(df):
        return BatchCodeTableFactory._createManufacturerByBatchCodeTable(df).rename(columns = {"VAX_MANU": "COMPANY"})


In [ ]:
import unittest

In [ ]:
from pandas.testing import assert_frame_equal

class DataFrameFilterTest(unittest.TestCase):

    def test_filterBy(self):
        # Given
        dataFrameFilter = DataFrameFilter(
            VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
                [
                    {
                        'VAERSDATA': self.createDataFrame(
                            columns = ['DIED', 'L_THREAT', 'DISABLE'],
                            data = [  ['Y',    np.NaN,     np.NaN],
                                        [np.NaN, np.NaN,     'Y']],
                            index = [
                                "0916600",
                                "0916601"]),
                        'VAERSVAX': self.createDataFrame(
                            columns = ['VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
                            data = [  ['COVID19',  'MODERNA',  '037K20A', '1'],
                                        ['COVID19',  'MODERNA',  '025L20A', '1']],
                            index = [
                                "0916600",
                                "0916601"],
                            dtypes = {'VAX_DOSE_SERIES': "string"})
                    },
                    {
                            'VAERSDATA': self.createDataFrame(
                            columns = ['DIED', 'L_THREAT', 'DISABLE'],
                            data = [  [np.NaN, np.NaN,     np.NaN],
                                        [np.NaN, np.NaN,     'Y']],
                                index = [
                                "1996873",
                                "1996874"]),
                            'VAERSVAX': self.createDataFrame(
                                columns = ['VAX_TYPE', 'VAX_MANU',         'VAX_LOT', 'VAX_DOSE_SERIES'],
                                data = [  ['HPV9',     'MERCK & CO. INC.', 'R017624', 'UNK'],
                                        ['COVID19',  'MODERNA',          '025L20A', '1']],
                                index = [
                                    "1996873",
                                    "1996874"],
                                dtypes = {'VAX_DOSE_SERIES': "string"})
                        }
                ]))
            
        # When
        dataFrame = dataFrameFilter.filterBy(manufacturer = "MODERNA", dose = '1')
        
        # Then
        dataFrameExpected = self.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE',  'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
            data = [  ['Y',     np.NaN,    np.NaN,     'COVID19',  'MODERNA',  '037K20A', '1'],
                      [np.NaN,  np.NaN,    'Y',        'COVID19',  'MODERNA',  '025L20A', '1'],
                      [np.NaN, np.NaN,     'Y',        'COVID19',  'MODERNA',  '025L20A', '1']],
            index = [
                "0916600",
                "0916601",
                "1996874"],
            dtypes = {'VAX_DOSE_SERIES': "string"})
        assert_frame_equal(dataFrame, dataFrameExpected, check_dtype = False)

    def test_filterForSevereEffects(self):
        # Given
        dataFrameFilter = DataFrameFilter(
            VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
                [
                    {
                        'VAERSDATA': self.createDataFrame(
                            columns = ['DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT'],
                            data = [  ['Y',    'Y',        np.NaN,    'Y',        'Y'],
                                        [np.NaN, np.NaN,     'Y',       np.NaN,     'Y']],
                            index = [
                                "0916600",
                                "0916601"]),
                        'VAERSVAX': self.createDataFrame(
                            columns = ['VAX_TYPE', 'VAX_MANU',        'VAX_LOT', 'VAX_DOSE_SERIES'],
                            data = [  ['COVID19',  'MODERNA',         '037K20A', '1'],
                                        ['COVID19',  'PFIZER\BIONTECH', '025L20A', '1']],
                            index = [
                                "0916600",
                                "0916601"],
                            dtypes = {'VAX_DOSE_SERIES': "string"})
                    }
                ]))

        # When
        dataFrame = dataFrameFilter.filterForSevereEffects(dose = '1')
        
        # Then
        dataFrameExpected = self.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT', 'VAX_TYPE', 'VAX_MANU',        'VAX_LOT', 'VAX_DOSE_SERIES'],
            data = [  ['Y',    'Y',        np.NaN,    'Y',        'Y',        'COVID19',  'MODERNA',         '037K20A', '1'],
                      [np.NaN,  np.NaN,    'Y',       np.NaN,     'Y',        'COVID19',  'PFIZER\BIONTECH', '025L20A', '1']],
            index = [
                "0916600",
                "0916601"],
            dtypes = {'VAX_DOSE_SERIES': "string"})
        assert_frame_equal(dataFrame, dataFrameExpected, check_dtype = False)

    def test_createAndFilterDataFrameFromDescrsWithFirstDose(self):
        # Given
        dataFrameFilter = DataFrameFilter(
            VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
                [
                    {
                        'VAERSDATA': self.createDataFrame(
                            columns = ['DIED', 'L_THREAT', 'DISABLE'],
                            data = [  ['Y',    np.NaN,      np.NaN]],
                            index = [
                                "1048786"]),
                        'VAERSVAX': self.createDataFrame(
                            columns = ['VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
                            data = [  ['COVID19',  'MODERNA',  '016M20A', '2'],
                                        ['COVID19',  'MODERNA',  '030L20A', '1']],
                            index = [
                                "1048786",
                                "1048786"],
                            dtypes = {'VAX_DOSE_SERIES': "string"})
                    }
                ]))
            
        # When
        dataFrame = dataFrameFilter.filterBy(manufacturer = "MODERNA", dose = '1')
        
        # Then
        dataFrameExpected = self.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE',  'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
            data = [  ['Y',     np.NaN,    np.NaN,     'COVID19',  'MODERNA',  '030L20A',  '1']],
            index = [
                "1048786"],
            dtypes = {'VAX_DOSE_SERIES': "string"})
        assert_frame_equal(dataFrame, dataFrameExpected, check_dtype = False)

    def test_createAndFilterDataFrameFromDescrsWithSecondDose(self):
        # Given
        dataFrameFilter = DataFrameFilter(
            VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
                [
                    {
                        'VAERSDATA': self.createDataFrame(
                            columns = ['DIED', 'L_THREAT', 'DISABLE'],
                            data = [  ['Y',    np.NaN,     np.NaN]],
                            index = [
                                "1048786"]),
                        'VAERSVAX': self.createDataFrame(
                            columns = ['VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
                            data = [  ['COVID19',  'MODERNA',  '016M20A',  '2'],
                                        ['COVID19',  'MODERNA',  '030L20A',  '1']],
                            index = [
                                "1048786",
                                "1048786"],
                            dtypes = {'VAX_DOSE_SERIES': "string"})
                    }
                ]))

        # When
        dataFrame = dataFrameFilter.filterBy(manufacturer = "MODERNA", dose = '2')
        
        # Then
        dataFrameExpected = self.createDataFrame(
            columns = ['DIED', 'L_THREAT', 'DISABLE',  'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
            data = [  ['Y',     np.NaN,    np.NaN,     'COVID19',  'MODERNA',  '016M20A',  '2']],
            index = [
                "1048786"],
            dtypes = {'VAX_DOSE_SERIES': "string"})
        assert_frame_equal(dataFrame, dataFrameExpected, check_dtype = False)

    def createDataFrame(self, index, columns, data, dtypes = {}):
        return pd.DataFrame(index = index, columns = columns, data = data).astype(dtypes)


In [ ]:
from pandas.testing import assert_frame_equal

class BatchCodeTableFactoryTest(unittest.TestCase):

    def test_createSevereEffectsBatchCodeTable(self):
        # Given
        batchCodeTableFactory = BatchCodeTableFactory(
            VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
                [
                    {
                        'VAERSDATA': self.createDataFrame(
                            columns = ['DIED', 'L_THREAT', 'DISABLE', 'HOSPITAL', 'ER_VISIT'],
                            data = [  ['Y',    'Y',        np.NaN,    'Y',        'Y'],
                                        [np.NaN, np.NaN,     'Y',       np.NaN,     'Y']],
                            index = [
                                "0916600",
                                "0916601"]),
                        'VAERSVAX': self.createDataFrame(
                            columns = ['VAX_TYPE', 'VAX_MANU',        'VAX_LOT', 'VAX_DOSE_SERIES'],
                            data = [  ['COVID19',  'MODERNA',         '037K20A', '1'],
                                        ['COVID19',  'PFIZER\BIONTECH', '025L20A', '1']],
                            index = [
                                "0916600",
                                "0916601"],
                            dtypes = {'VAX_DOSE_SERIES': "string"})
                    }
                ]))

        # When
        batchCodeTable = batchCodeTableFactory.createSevereEffectsBatchCodeTable('1')

        # Then
        batchCodeTableExpected = pd.DataFrame(
            data = {
                'ADRs': [1, 1],
                'DEATHS': [1, 0],
                'DISABILITIES': [0, 1],
                'LIFE THREATENING ILLNESSES': [1, 0],
                'HOSPITALISATIONS': [1, 0],
                'EMERGENCY ROOM OR DOCTOR VISITS': [1, 1],
                'COMPANY': ['MODERNA', 'PFIZER\BIONTECH']
            },
            index = pd.Index(['037K20A', '025L20A'], name='VAX_LOT'))
        assert_frame_equal(batchCodeTable, batchCodeTableExpected, check_dtype = False)

    def test_createBatchCodeTable2(self):
        dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
            [
               {
                   'VAERSDATA': self.createDataFrame(
                       columns = ['DIED', 'L_THREAT', 'DISABLE'],
                       data = [  ['Y',    np.NaN,     np.NaN],
                                 [np.NaN, np.NaN,     'Y']],
                       index = [
                           "0916600",
                           "0916601"]),
                   'VAERSVAX': self.createDataFrame(
                       columns = ['VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES'],
                       data = [  ['COVID19',  'MODERNA',  '037K20A', '1'],
                                 ['COVID19',  'MODERNA',  '025L20A', '1']],
                       index = [
                           "0916600",
                           "0916601"],
                       dtypes = {'VAX_DOSE_SERIES': "string"})
               },
               {
                    'VAERSDATA': self.createDataFrame(
                       columns = ['DIED', 'L_THREAT', 'DISABLE'],
                       data = [  [np.NaN,  np.NaN,    np.NaN],
                                 [np.NaN,  np.NaN,    'Y']],
                       index = [
                           "1996873",
                           "1996874"]),
                    'VAERSVAX': self.createDataFrame(
                        columns = ['VAX_TYPE', 'VAX_MANU',         'VAX_LOT', 'VAX_DOSE_SERIES'],
                        data = [  ['HPV9',     'MERCK & CO. INC.', 'R017624', 'UNK'],
                                  ['COVID19',  'MODERNA',          '025L20A', '1']],
                        index = [
                            "1996873",
                            "1996874"],
                        dtypes = {'VAX_DOSE_SERIES': "string"})
                }
            ])
        self._test_createBatchCodeTable(dataFrame, "MODERNA", '1')

    def test_createBatchCodeTable(self):
        dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(
            VaersDescrReader("test/VAERS").readAllVaersDescrs())
        self._test_createBatchCodeTable(dataFrame, "MODERNA", '1')

    def _test_createBatchCodeTable(self, dataFrame, manufacturer, dose):
        # Given
        batchCodeTableFactory = BatchCodeTableFactory(dataFrame)

        # When
        batchCodeTable = batchCodeTableFactory.createBatchCodeTable(manufacturer, dose)

        # Then
        batchCodeTableExpected = pd.DataFrame(
            data = {
                'ADRs': [2, 1],
                'DEATHS': [0, 1],
                'DISABILITIES': [2, 0],
                'LIFE THREATENING ILLNESSES': [0, 0]
            },
            index = pd.Index(['025L20A', '037K20A'], name = 'VAX_LOT'))
        assert_frame_equal(batchCodeTable, batchCodeTableExpected, check_dtype = False)

    def createDataFrame(self, index, columns, data, dtypes = {}):
        return pd.DataFrame(index = index, columns = columns, data = data).astype(dtypes)


In [ ]:
unittest.main(argv = [''], verbosity = 2, exit = False)

In [ ]:
def saveBatchCodeTable(manufacturer, excelFile):
    vaersDescrs = VaersDescrReader("VAERS").readAllVaersDescrs()
    dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(vaersDescrs)
    batchCodeTableFactory = BatchCodeTableFactory(dataFrame)
    batchCodeTable = batchCodeTableFactory.createBatchCodeTable(manufacturer = manufacturer, dose = '1')
    display(manufacturer, batchCodeTable)
    batchCodeTable.to_excel(excelFile)

In [ ]:
saveBatchCodeTable("MODERNA", "results/moderna.xlsx")
saveBatchCodeTable("PFIZER\BIONTECH", "results/pfizer.xlsx")
saveBatchCodeTable("JANSSEN", "results/janssen.xlsx")

In [ ]:
def saveSevereEffectsBatchCodeTable(excelFile):
    vaersDescrs = VaersDescrReader("VAERS").readAllVaersDescrs()
    dataFrame = VaersDescr2DataFrameConverter.createDataFrameFromDescrs(vaersDescrs)
    batchCodeTableFactory = BatchCodeTableFactory(dataFrame)
    severeEffectsBatchCodeTable = batchCodeTableFactory.createSevereEffectsBatchCodeTable(dose = '1')
    display('severeEffectsBatchCodeTable', severeEffectsBatchCodeTable)
    severeEffectsBatchCodeTable.to_excel(excelFile)

In [ ]:
saveSevereEffectsBatchCodeTable('results/severeEffects.xlsx')